In [90]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
import re

In [91]:
# import parser
import spacy 
from spacy.matcher import PhraseMatcher 
nlp = spacy.load('en_core_web_sm')
matcher = PhraseMatcher(nlp.vocab)

# terms would be each list in the dict

In [93]:
import json 

# with open ('2020_may30_lexiconV1.json') as f1:
#     dic = json.load(f1)
with open ('2020_july_lexiconV4_61.json') as f1:
    dic = json.load(f1)

In [94]:
print(sorted(dic))

['covid', 'economic', 'education', 'environmental', 'foreign policy', 'governance', 'health', 'immigration', 'military', 'safety', 'social cultural', 'social programs']


In [95]:
# stopword = stopwords.words('english')

In [96]:
df1 = pd.read_csv('2020july21_FBtest_1500_LabelResult.csv')
# df1 = pd.read_excel('Facebook Ads Analysis.xlsx') # this data is coming from the facebook ad 2016 (the )

In [97]:
df1.columns

Index(['text', 'Gold Label', 'words', 'm_label'], dtype='object')

In [98]:
df1 = df1[['Gold Label', 'words', 'm_label']]

In [99]:
# filter the stop words in the 

def filter_lexicon(lexicon):
    topics= lexicon.keys()
    for t in topics:
        ls=[]
        for phrase in lexicon[t]:
            phrase = ' '.join([w.lower() for w in phrase.split()])# if w.lower() not in stopword])
            ls.append(phrase)
            # phrase = re.sub("[^\w\s]",' ',phrase)
             
                              
        lexicon[t]=ls
    return lexicon

In [100]:
dic1 = filter_lexicon(dic)

In [101]:
# pred_label = ['economic', 'environmental', 'foreign policy', 'governance', 
#               'immigration', 'military', 'safety','social programs', 
#               'social cultural']
n=len(df1)


pred_label = dic1.keys()


data1 = {x:np.zeros(n,dtype=int) for x in pred_label}

#data1 = {x:'' for x in pred_label}

tabel1 = pd.DataFrame(data1)

df2 = pd.concat([df1,tabel1],axis=1)

# df1.tail(10)

In [102]:
# find the words in each message
topics=dic1.keys()
    
for index, row in df2.iterrows():
    if isinstance(row['words'],str) is False:
        df2.loc[index,'words']= 'no words'
    if isinstance(row['words'],str): 
        # print(row['words'])
        words = row['words'].split(',')        
        for t in topics:            
            terms = dic1[t]
            if set(words)&set(terms):  
                a = list(set(words)&set(terms))
                # row[t]=','.join(set(words)&set(terms))
                # df2.loc[index,t]=','.join(set(words)&set(terms))
                df2.loc[index,t]= ','.join(a)

                
            

In [103]:
df2.tail(10)

,Gold Label,words,m_label,covid,economic,education,environmental,foreign policy,governance,health,immigration,military,safety,social cultural,social programs
1489,"social cultural,covid","covid 19,covid,color","covid,social cultural","covid,covid 19",0,0,0,0,0,0,0,0,0,color,0
1490,no topic,refugee,immigration,0,0,0,0,0,0,0,refugee,0,0,0,0
1491,no topic,no words,no topic,0,0,0,0,0,0,0,0,0,0,0,0
1492,no topic,no words,no topic,0,0,0,0,0,0,0,0,0,0,0,0
1493,"immigration,governance",supreme court,governance,0,0,0,0,0,supreme court,0,0,0,0,0,0
1494,no topic,no words,no topic,0,0,0,0,0,0,0,0,0,0,0,0
1495,no topic,no words,no topic,0,0,0,0,0,0,0,0,0,0,0,0
1496,no topic,no words,no topic,0,0,0,0,0,0,0,0,0,0,0,0
1497,no topic,no words,no topic,0,0,0,0,0,0,0,0,0,0,0,0
1498,no topic,no words,no topic,0,0,0,0,0,0,0,0,0,0,0,0


In [104]:
df3 = df2

In [105]:
for index, row in df3.iterrows():
    
    if len(row['Gold Label'].split(',')) ==1 and row['Gold Label']!= 'no topic':
        if row['Gold Label'].strip() == row['m_label'].strip():
            label=row['Gold Label'].strip()
            df3.loc[index,label]= 'correct label'
        
    elif len(row['Gold Label'].split(','))>1:
        if set(row['Gold Label'].split(','))&set(row['m_label'].split(',')):
            a = set(row['Gold Label'].split(','))&set(row['m_label'].split(','))
            #print(a)
            ls = list(a)
            for w in a :
                # print(w)
                df3.loc[index,w]= 'correct label'
    
    elif row['Gold Label']== 'no topic' and row['words']!= "no words":
        b = set(row['m_label'].split(','))
        print('false: '+ str(b))
                
    elif row['Gold Label']!= 'no topic' and row['words']== "no words":
        pass 
    elif row['Gold Label'] == 'no topic' and row['words']== "no words":
        pass
        
    else:
        print('???')

false: {'economic'}
false: {'economic'}
false: {'economic'}
false: {'education'}
false: {'education'}
false: {'social programs'}
false: {'education'}
false: {'social cultural'}
false: {'economic'}
false: {'social cultural'}
false: {'economic'}
false: {'economic'}
false: {'economic'}
false: {'economic'}
false: {'social cultural'}
false: {'social cultural'}
false: {'social cultural'}
false: {'social programs'}
false: {'health', 'social programs'}
false: {'military'}
false: {'education'}
false: {'social cultural'}
false: {'economic'}
false: {'economic'}
false: {'economic'}
false: {'covid'}
false: {'economic', 'social programs', 'health'}
false: {'health'}
false: {'safety'}
false: {'economic'}
false: {'economic'}
false: {'social cultural'}
false: {'governance'}
false: {'education', 'social programs'}
false: {'social cultural'}
false: {'economic'}
false: {'economic'}
false: {'economic'}
false: {'economic'}
false: {'social cultural'}
false: {'social cultural'}
false: {'governance'}
false: {'

In [106]:
df3.loc[13:20,:]

,Gold Label,words,m_label,covid,economic,education,environmental,foreign policy,governance,health,immigration,military,safety,social cultural,social programs
13,economic,"financial protection,banks,consumers,protect c...",economic,0,correct label,0,0,0,0,0,0,0,0,0,0
14,education,"student,college,debt","education,social programs",0,0,student,0,0,0,0,0,0,0,0,"college,student,debt"
15,no topic,no words,no topic,0,0,0,0,0,0,0,0,0,0,0,0
16,no topic,no words,no topic,0,0,0,0,0,0,0,0,0,0,0,0
17,no topic,no words,no topic,0,0,0,0,0,0,0,0,0,0,0,0
18,social programs,social security,social programs,0,0,0,0,0,0,0,0,0,0,0,correct label
19,social programs,social security,social programs,0,0,0,0,0,0,0,0,0,0,0,correct label
20,governance,self funding billionaire,governance,0,0,0,0,0,correct label,0,0,0,0,0,0


In [84]:
# df3.fillna(0,inplace=True)

In [107]:
df3.to_csv('2020july21_FB1500_FindMiss.csv',index=False)

In [108]:
remove_v4 = {}
for c in dic1.keys():
    df4 = df3[[c]]
    ls = df4[c].loc[(df4[c]!=0) & (df4[c]!='correct label')].tolist()
    ls = [x for x in ls if isinstance(x,str)]
    wls = ','.join(ls)
    wls = list(set(wls.split(',')))
    d = {c:wls}
    remove_v4.update(d)

In [109]:
remove_v4

{'covid': ['covid', 'covid 19'],
 'economic': ['middle class',
  'ceos',
  'jobs',
  'businessman',
  'wealth',
  'american worker',
  'workers',
  'prosperity',
  'american workers',
  'business leaders',
  'treasurer',
  'economy'],
 'education': ['school',
  'education',
  'public education',
  'student',
  'public schools',
  'loans',
  'teachers'],
 'environmental': ['climate', 'environment', 'climate change', 'ethanol'],
 'foreign policy': ['muslim ban',
  'foreign',
  'alliances',
  'south korea',
  'usmca',
  'aipac',
  'israel',
  'foreign money'],
 'governance': ['blasey ford',
  'political power',
  'due process',
  'impeachment',
  'constitution'],
 'health': ['aids', 'health care', 'health'],
 'immigration': ['refugee', 'border', 'detention'],
 'military': ['veterans affairs', 'troops', 'base'],
 'safety': ['assault', 'criminal', 'drugs', 'people dead'],
 'social cultural': ['black',
  'disabilities',
  'racist',
  'sexual orientation',
  'muslim',
  'people of color',
  '

In [110]:
with open ('remove_V461.json','w') as f1:
    json.dump(remove_v4,f1,sort_keys=True, indent=4)

In [111]:
df3[df3['education']!=0].shape

(55, 15)